<a href="https://colab.research.google.com/github/TheoLpr/NLI_study/blob/main/Natural_language_inference_roberta_probas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library setup

In [ ]:
!pip3 install -q --upgrade transformers
!pip3 install -q --upgrade datasets
!pip3 install -q --upgrade evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Subset

import numpy as np
from tqdm import tqdm

Use `AutoModelFor...` as this handles potential differences in model architectures in the background. Helpful so you can run many different models using practically the same code.
Similar story for `AutoTokenizer`.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
pretrained_name_or_path = "roberta-base"
num_classes = 3  # {"entailment", "neutral", "contradiction"}
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
MAX_LENGTH = 128
NUM_EPOCHS = 1
LEARNING_RATE = 2e-6
BATCH_SIZE = 4
VALIDATE_EVERY_N_EXAMPLES = 10_000
# strings that should not be used in model, but might be useful for connecting predictions with examples
RAW_DATA_KEYS = {"premise", "hypothesis"}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_name_or_path,
                                                           num_labels=num_classes).to(DEVICE)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should pr

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def tokenize_function(data_to_tokenize):
    return tokenizer(list(zip(data_to_tokenize["premise"],
                              data_to_tokenize["hypothesis"])),
                     max_length=MAX_LENGTH,
                     padding="max_length",
                     truncation=True,
                     return_tensors="pt")

##DATA LOADING


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

train_data=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/train_snli_probas.csv")
test_data=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/test_snli_probas.csv")
val_data=pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/val_snli_probas.csv")

train_data_probas=train_data[["probas_label","index"]]
test_data_probas=test_data[["probas_label","index","gold_label"]]
val_data_probas=val_data[["probas_label","index","gold_label"]]

In [ ]:
val_data

,Unnamed: 0,probas_label,index,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,0,[0.19880766 0.80119234 0. ],0,1,( ( Two women ) ( ( are ( embracing ( while ( ...,( ( The sisters ) ( ( are ( ( hugging goodbye ...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP ...,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,4705552913.jpg#2,4705552913.jpg#2r1n,1,0,1,1,1
1,1,[1. 0. 0.],1,0,( ( Two women ) ( ( are ( embracing ( while ( ...,( ( Two woman ) ( ( are ( holding packages ) )...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (CD Two) (NN woman)) (VP (VBP are...,Two women are embracing while holding to go pa...,Two woman are holding packages.,4705552913.jpg#2,4705552913.jpg#2r1e,0,0,0,0,0
2,2,[0. 0. 1.],2,2,( ( Two women ) ( ( are ( embracing ( while ( ...,( ( The men ) ( ( are ( fighting ( outside ( a...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (DT The) (NNS men)) (VP (VBP are)...,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,4705552913.jpg#2,4705552913.jpg#2r1c,2,2,2,2,2
3,3,[1. 0. 0.],3,0,( ( ( Two ( young children ) ) ( in ( ( ( ( ( ...,( ( ( Two kids ) ( in ( numbered jerseys ) ) )...,(ROOT (S (NP (NP (CD Two) (JJ young) (NNS chil...,(ROOT (S (NP (NP (CD Two) (NNS kids)) (PP (IN ...,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,2407214681.jpg#0,2407214681.jpg#0r1e,0,0,0,0,0
4,4,[0.39639431 0.60360569 0. ],4,1,( ( ( Two ( young children ) ) ( in ( ( ( ( ( ...,( ( ( Two kids ) ( at ( a ballgame ) ) ) ( ( w...,(ROOT (S (NP (NP (CD Two) (JJ young) (NNS chil...,(ROOT (S (NP (NP (CD Two) (NNS kids)) (PP (IN ...,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,2407214681.jpg#0,2407214681.jpg#0r1n,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,9981,[1. 0. 0.],9994,0,( ( ( A ( small girl ) ) ( wearing ( a ( pink ...,( ( The carousel ) ( ( is moving ) . ) ),(ROOT (S (NP (NP (DT A) (JJ small) (NN girl)) ...,(ROOT (S (NP (DT The) (NN carousel)) (VP (VBZ ...,A small girl wearing a pink jacket is riding o...,The carousel is moving.,77063034.jpg#3,77063034.jpg#3r1e,0,0,0,0,0
9982,9982,[0. 0. 1.],9996,2,( ( ( A ( small girl ) ) ( wearing ( a ( pink ...,( ( The girl ) ( ( is ( moving ( at ( ( the sp...,(ROOT (S (NP (NP (DT A) (JJ small) (NN girl)) ...,(ROOT (S (NP (DT The) (NN girl)) (VP (VBZ is) ...,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.,77063034.jpg#3,77063034.jpg#3r1c,2,2,2,2,2
9983,9983,[1. 0. 0.],9997,0,( ( ( A ( young girl ) ) ( with ( ( ( ( ( ( bl...,( People ( in ( a ( water fountain ) ) ) ),(ROOT (NP (NP (DT A) (JJ young) (NN girl)) (PP...,(ROOT (NP (NP (NNS People)) (PP (IN in) (NP (D...,A young girl with blue and pink ribbons in her...,People in a water fountain,4805835848.jpg#0,4805835848.jpg#0r1e,0,0,0,0,0
9984,9984,[0.20435007 0.20511877 0.59053116],9998,2,( ( ( A ( young girl ) ) ( with ( ( ( ( ( ( bl...,( ( ( A ( young girl ) ) knits ) ( a sweater ) ),(ROOT (NP (NP (DT A) (JJ young) (NN girl)) (PP...,(ROOT (NP (NP (DT A) (JJ young) (NN girl)) (NP...,A young girl with blue and pink ribbons in her...,A young girl knits a sweater,4805835848.jpg#0,4805835848.jpg#0r1c,2,2,0,2,1


In [ ]:
train_data["index"]

0            11
1            16
2            26
3            29
4            34
          ...  
36771    550093
36772    550095
36773    550097
36774    550124
36775    550134
Name: index, Length: 36776, dtype: int64

In [ ]:
import datasets
data=datasets.load_dataset("snli")

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
val_data_probas

,probas_label,index,gold_label
0,[0.19880766 0.80119234 0. ],0,1
1,[1. 0. 0.],1,0
2,[0. 0. 1.],2,2
3,[1. 0. 0.],3,0
4,[0.39639431 0.60360569 0. ],4,1
...,...,...,...
9981,[1. 0. 0.],9994,0
9982,[0. 0. 1.],9996,2
9983,[1. 0. 0.],9997,0
9984,[0.20435007 0.20511877 0.59053116],9998,2


In [ ]:
# change the type of label because it has been loaded as a string

l_train_probs=[]
for i in range(len(train_data_probas["probas_label"])):
  l = [float(x.strip(' []')) for x in train_data_probas["probas_label"][i].split(maxsplit=2)]
  l_train_probs.append(l)


l_val_probs=[]
for elt in val_data_probas["probas_label"]:
  l = [float(x.strip(' []')) for x in elt.split(maxsplit=2)]
  l_val_probs.append(l)

l_test_probs=[]
for elt in test_data_probas["probas_label"]:
  l = [float(x.strip(' []')) for x in elt.split(maxsplit=2)]
  l_test_probs.append(l)

In [ ]:
# Drop examples with invalid labels
train_data = data["train"].select(list(train_data_probas["index"])).remove_columns("label").add_column("labels",l_train_probs)
val_data = data["validation"].select(list(val_data_probas["index"])).remove_columns("label").add_column("labels",l_val_probs)
test_data = data["test"].select(list(test_data_probas["index"])).remove_columns("label").add_column("labels",l_test_probs)

real_train_labels=data["train"].select(list(train_data_probas["index"]))["label"]
real_val_labels=data["validation"].select(list(val_data_probas["index"]))["label"]
real_test_labels=data["test"].select(list(test_data_probas["index"]))["label"]

train_data = train_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
val_data = val_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
test_data = test_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")

Map:   0%|          | 0/36776 [00:00<?, ? examples/s]

Map:   0%|          | 0/9986 [00:00<?, ? examples/s]

Map:   0%|          | 0/9990 [00:00<?, ? examples/s]

In [ ]:
import sklearn.metrics
from sklearn.metrics import mean_squared_error

In [ ]:
NUM_SUBSETS = (len(train_data) + VALIDATE_EVERY_N_EXAMPLES - 1) // VALIDATE_EVERY_N_EXAMPLES

##Training part

In [ ]:
for idx_epoch in range(NUM_EPOCHS):
    rand_indices = torch.randperm(len(train_data))
    train_loss, num_batches = 0.0, 0

    for idx_subset in range(NUM_SUBSETS):
        s_sub, e_sub = idx_subset * VALIDATE_EVERY_N_EXAMPLES, (idx_subset + 1) * VALIDATE_EVERY_N_EXAMPLES

        # Training loop
        model.train()

        # DataLoader handles things such as batching data,
        # Subset creates a smaller data subset
        for curr_batch in tqdm(DataLoader(Subset(train_data, rand_indices[s_sub: e_sub].tolist()),
                                          batch_size=BATCH_SIZE)):
           # for _k in curr_batch:
            #  if _k not in RAW_DATA_KEYS:
             #   print("Numéro ",z,"\n",curr_batch[_k])
              #  print(curr_batch[_k].to(DEVICE))
               # z+=1

            batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}

            res = model(**batch_on_device)

            loss = res.loss
            train_loss += float(loss)
            num_batches += 1

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f"Training loss: {train_loss / max(1, num_batches):.4f}")

        # Validation loop
        val_preds = []

        # .eval turns off dropout layers in model
        model.eval()

        # torch.inference_mode() makes sure gradients are not being computed
        # (they are only required during training and require a lot of memory)
        with torch.inference_mode():
            for curr_batch in tqdm(DataLoader(val_data, batch_size=BATCH_SIZE)):
                batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                                   if _k not in RAW_DATA_KEYS}
                res = model(**batch_on_device)

                # logits = class scores; normalize them to get probabilities
                probas = torch.softmax(res.logits, dim=-1)
                preds = torch.argmax(probas, dim=-1).cpu()
                val_preds.append(preds)

        val_preds = torch.cat(val_preds)
        # TODO: compute whatever metric you wish
        accuracy = torch.sum(torch.sum(torch.Tensor(real_val_labels)==val_preds)) / len(real_val_labels)
        print(f"Validation accuracy: {accuracy:.4f}")

        # TODO: validate if this metric is better than the current best,
        # save model if it is the best one, etc.
        # ...

100%|██████████| 2500/2500 [05:04<00:00,  8.22it/s]


Training loss: 0.4642


100%|██████████| 2497/2497 [01:13<00:00, 33.81it/s]


Validation accuracy: 0.8178


100%|██████████| 2500/2500 [05:09<00:00,  8.08it/s]


...

Validation accuracy: 0.8495


100%|██████████| 1694/1694 [03:29<00:00,  8.09it/s]


Training loss: 0.3742


100%|██████████| 2497/2497 [01:13<00:00, 33.95it/s]

Validation accuracy: 0.8568


In [ ]:
val_preds[:10]


tensor([1, 0, 2, 0, 1, 2, 2, 1, 0, 0])

In [ ]:

val_preds = []

# .eval turns off dropout layers in model
model.eval()

# torch.inference_mode() makes sure gradients are not being computed
# (they are only required during training and require a lot of memory)
with torch.inference_mode():
    for curr_batch in tqdm(DataLoader(val_data, batch_size=BATCH_SIZE)):
        batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}
        res = model(**batch_on_device)

        # logits = class scores; normalize them to get probabilities
        probas = torch.softmax(res.logits, dim=-1)
        preds = torch.argmax(probas, dim=-1).cpu()
        val_preds.append(preds)

val_preds = torch.cat(val_preds)
# TODO: compute whatever metric you wish
accuracy = torch.sum(val_preds == torch.Tensor(real_val_labels)) / len(real_val_labels)
print(f"Validation accuracy: {accuracy:.4f}")

100%|██████████| 2497/2497 [01:15<00:00, 33.15it/s]


Validation accuracy: 0.8568


In [ ]:
len(real_train_labels)

36776

In [ ]:
val_data

Dataset({
    features: ['premise', 'hypothesis', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 9986
})

In [ ]:
val_data_probas

,probas_label,index,gold_label
0,[0.19880766 0.80119234 0. ],0,1
1,[1. 0. 0.],1,0
2,[0. 0. 1.],2,2
3,[1. 0. 0.],3,0
4,[0.39639431 0.60360569 0. ],4,1
...,...,...,...
9981,[1. 0. 0.],9994,0
9982,[0. 0. 1.],9996,2
9983,[1. 0. 0.],9997,0
9984,[0.20435007 0.20511877 0.59053116],9998,2


Use the HuggingFace ecosystem's built-in saving function to save your model: `<model>.save_pretrained(<path>)`. This path can be used in `.from_pretrained(<path>)` to reload the model later.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
torch.save(model, "/content/gdrive/MyDrive/Colab Notebooks/model_probas.pth")

In [ ]:
model2=torch.load("/content/gdrive/MyDrive/Colab Notebooks/model_probas.pth")

In [ ]:
train_data

Dataset({
    features: ['premise', 'hypothesis', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 36776
})

In [ ]:
train_data2_inv=train_data.rename_column("hypothesis","temp").rename_column("premise","hypothesis").rename_column("temp","premise").remove_columns(["labels","input_ids","attention_mask"]).map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
test_data2_inv=test_data.map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")
val_data2_inv=val_data.rename_column("hypothesis","temp").rename_column("premise","hypothesis").rename_column("temp","premise").remove_columns(["labels","input_ids","attention_mask"]).map(tokenize_function, batched=True, batch_size=10000, load_from_cache_file=False).with_format("torch")


Map:   0%|          | 0/36776 [00:00<?, ? examples/s]

Map:   0%|          | 0/9990 [00:00<?, ? examples/s]

Map:   0%|          | 0/9986 [00:00<?, ? examples/s]

In [ ]:
train_data2_inv

Dataset({
    features: ['hypothesis', 'premise', 'input_ids', 'attention_mask'],
    num_rows: 36776
})

##Prediction of reverse data

In [ ]:

# Validation loop
train_preds_inv = []
test_preds_inv = []
val_preds_inv = []

# .eval turns off dropout layers in model
model.eval()

# torch.inference_mode() makes sure gradients are not being computed
# (they are only required during training and require a lot of memory)
with torch.inference_mode():
    for curr_batch in tqdm(DataLoader(val_data2_inv, batch_size=1)):
        batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}
        res = model(**batch_on_device)

        # logits = class scores; normalize them to get probabilities
        probas = torch.softmax(res.logits, dim=-1)
        val_preds_inv.append(probas)

with torch.inference_mode():
    for curr_batch in tqdm(DataLoader(train_data2_inv, batch_size=1)):
        batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}
        res = model(**batch_on_device)

        # logits = class scores; normalize them to get probabilities
        probas = torch.softmax(res.logits, dim=-1)
        train_preds_inv.append(probas)

with torch.inference_mode():
    for curr_batch in tqdm(DataLoader(test_data2_inv, batch_size=1)):
        batch_on_device = {_k: curr_batch[_k].to(DEVICE) for _k in curr_batch
                            if _k not in RAW_DATA_KEYS}
        res = model(**batch_on_device)

        # logits = class scores; normalize them to get probabilities
        probas = torch.softmax(res.logits, dim=-1)
        test_preds_inv.append(probas)

100%|██████████| 9990/9990 [01:56<00:00, 85.88it/s]


In [ ]:
val_preds_inv

In [ ]:
from sklearn import metrics

In [ ]:
real_val_labels

In [ ]:
actual=list(map(int,real_val_labels))

predicted=list(map(int,val_preds))

confusion_matrix = metrics.confusion_matrix(actual, predicted)

#virer les tirets -> crééer une 2ème conf matrix
confusion_matrix2=np.zeros((4,3))
i=0
for ligne in confusion_matrix:
  confusion_matrix2[i]=[ligne[1],ligne[2],ligne[3]]
  i+=1

array([[  48.,   73.,   34.],
       [2939.,  262.,  124.],
       [ 228., 2714.,  289.],
       [ 107.,  265., 2903.]])

In [ ]:
print(confusion_matrix2[1:4])


[[2939.  262.  124.]
 [ 228. 2714.  289.]
 [ 107.  265. 2903.]]


##Collect predictions as a list (easier to process)

In [ ]:
train_preds_inv2=[]
for i in range(len(train_preds_inv)):
  train_preds_inv2.append(train_preds_inv[i].tolist()[0])


test_preds_inv2=[]
for i in range(len(test_preds_inv)):
  test_preds_inv2.append(test_preds_inv[i].tolist()[0])


val_preds_inv2=[]
for i in range(len(val_preds_inv)):
  val_preds_inv2.append(val_preds_inv[i].tolist()[0])

In [ ]:
for elt in test_preds_inv2 :
  print(sum(elt))

In [ ]:
train_data2_inv=train_data2_inv.add_column("label",train_preds_inv2)
val_data2_inv=val_data2_inv.add_column("label",val_preds_inv2)
test_data2_inv=test_data2_inv.add_column("label",test_preds_inv2)

In [ ]:
train_data2_inv.save_to_disk("/content/gdrive/MyDrive/Colab Notebooks/train_data_inv_probas")
val_data2_inv.save_to_disk("/content/gdrive/MyDrive/Colab Notebooks/val_data_inv_probas")
test_data2_inv.save_to_disk("/content/gdrive/MyDrive/Colab Notebooks/test_data_inv_probas")


Saving the dataset (0/1 shards):   0%|          | 0/36776 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9986 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9990 [00:00<?, ? examples/s]